## Generation of the GT for Training

In [2]:
import math
import sys
import time
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import spatial

from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map

from sphere import Sphere
from visualize import Visualize
from semantic_classes import SemanticClasses
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# NUSCENES LOADER
# ----------------------------------------------------------------

# export_ds = '/media/berlukas/Data2/datasets/nuscenes/processed/'
export_ds = '/media/scratch/berlukas/nuscenes'
sem_clouds_filename = f"{export_ds}/clouds1_val_bw50.npy"

print(f"Loading from sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/nuscenes/clouds1_val_bw50.npy
Shape of sem clouds is (500, 2, 100, 100)


In [2]:
# KITTI LOADER
# ----------------------------------------------------------------

# export_ds = '/media/scratch/berlukas/kitti/features50'
# export_ds = '/media/berlukas/Data/data/datasets/s2ae/cluster/val_kitti/'
export_ds = '/media/berlukas/SSD_1TB/s2ae/kitti/val/'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/berlukas/SSD_1TB/s2ae/kitti/val//clouds-08-rotated-val-180.npy
Shape of sem clouds is (500, 2, 100, 100)


In [2]:
# WAYMO LOADER
# ----------------------------------------------------------------

export_ds = '/media/scratch/berlukas/waymo/extracted/features_50'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/waymo/extracted/features_50/features02.npy
Shape of sem clouds is (500, 2, 100, 100)


In [2]:
# POSS LOADER
# ----------------------------------------------------------------

export_ds = '/media/scratch/berlukas/poss/features50'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))    
    
# Remove intensity channel
print(f"Shape of sem clouds before intensity removal is {sem_cloud_features.shape}")
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/poss/features50/clouds-05.npy
Shape of sem clouds before intensity removal is (500, 3, 100, 100)
Shape of sem clouds is (500, 2, 100, 100)


In [2]:
# USL LOADER
# ----------------------------------------------------------------

export_ds = '/media/scratch/berlukas/usl/features50'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
print(f"Shape of sem clouds before is {sem_cloud_features.shape}")
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/usl/features50/clouds-03.npy
Loading from sem clouds from /media/scratch/berlukas/usl/features50/clouds-12.npy
Shape of sem clouds before is (600, 3, 100, 100)
Shape of sem clouds is (600, 2, 100, 100)


In [3]:
# A2D2 LOADER
# ----------------------------------------------------------------

export_ds = '/media/berlukas/Data/data/datasets/s2ae/pc urban/processed/features50'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
print(f"Shape of sem clouds before is {sem_cloud_features.shape}")
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/berlukas/Data/data/datasets/s2ae/pc urban/processed/features50/clouds.npy
Shape of sem clouds before is (573, 3, 100, 100)
Shape of sem clouds is (573, 2, 100, 100)


In [ ]:
# A2D2 LOADER
# ----------------------------------------------------------------

export_ds = '/media/scratch/berlukas/a2d2/features50'
archives = os.listdir(export_ds)
sem_cloud_features = None
for archive in archives:
    sem_clouds_filename = f"{export_ds}/{archive}"
    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename)
    else:
        features = np.load(sem_clouds_filename)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
    
# Remove intensity channel
print(f"Shape of sem clouds before is {sem_cloud_features.shape}")
sem_cloud_features = sem_cloud_features[:, [0, 2], :, :]
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

In [4]:
n_clouds = sem_cloud_features.shape[0]
idx_sem = 1
classes = [None] * n_clouds
for i in tqdm(range(0, n_clouds)):
    sem_cloud = np.reshape(sem_cloud_features[i,:,:,:], (2, -1)).T    
    classes[i] = sem_cloud[:, idx_sem]
classes = np.array(classes[:])
print(f"shape of classes is {classes.shape}")
unique_classes = np.unique(classes)
n_unique_classes = len(unique_classes)
print(f"we have {n_unique_classes} classes in the dataset. That is:")
print(unique_classes)

  0%|          | 0/573 [00:00<?, ?it/s]

shape of classes is (573, 10000)
we have 21 classes in the dataset. That is:
[-1.  0.  1.  2.  3.  4.  6.  7.  8.  9. 10. 11. 13. 14. 15. 17. 18. 26.
 28. 29. 30.]


## Iterate over the features and generate the GT

In [5]:
n_features = sem_cloud_features.shape[0]
n_rows = sem_cloud_features.shape[2]
n_cols = sem_cloud_features.shape[3]
idx_range = 0
idx_sem = 1
max_range = 20

print(f'Iterating for {n_features} features and data of size: ({n_rows}/{n_cols})')
for feature in tqdm(range(n_features)):
    for i in range(n_rows):
        for j in range(n_cols):            
            cur_class = int(sem_cloud_features[feature, idx_sem, i, j])
#             mapped_class = SemanticClasses.map_nuscenes_label_old(cur_class)
#             mapped_class = SemanticClasses.map_nuscenes_label(cur_class)
#             mapped_class = SemanticClasses.map_waymo_to_nuscenes_label(cur_class)
            # mapped_class = SemanticClasses.map_kitti_to_nuscenes_label(cur_class)
#             mapped_class = SemanticClasses.map_poss_to_nuscenes_label(cur_class)
#             mapped_class = SemanticClasses.map_usl_to_nuscenes_label(cur_class)
            # mapped_class = SemanticClasses.map_a2d2_to_nuscenes_label(cur_class)
            mapped_class = SemanticClasses.map_pc_urban_to_nuscenes_label(cur_class)
            if (sem_cloud_features[feature, idx_range, i, j] <= max_range):
                sem_cloud_features[feature, idx_sem, i, j] = mapped_class
            else:
                sem_cloud_features[feature, idx_sem, i, j] = 0

Iterating for 573 features and data of size: (100/100)


  0%|          | 0/573 [00:00<?, ?it/s]

In [6]:
filename = 'sem_clouds_pc_urban_bw50'
sem_clouds_gt_filename = f'{export_ds}/{filename}.npy'
np.save(sem_clouds_gt_filename, sem_cloud_features)
print(f"Wrote features to {sem_clouds_gt_filename}")

# Create a tiny dataset
# n_process = 400
# sem_clouds_gt_tiny_filename = f'{export_ds}/{filename}_tiny.npy'
# np.save(sem_clouds_gt_tiny_filename, sem_cloud_features[0:n_process, :, :, :])
# print(f"Wrote tiny features to {sem_clouds_gt_tiny_filename}")

Wrote features to /media/berlukas/Data/data/datasets/s2ae/pc urban/processed/features50/sem_clouds_pc_urban_bw50.npy


In [5]:
# Remove a few samples for validation
n_clouds = sem_cloud_features.shape[0]
print(f'we have in total {n_clouds}')
n_train = n_clouds - 500
n_val = n_clouds - n_train
print(f'we will keep {n_train} for training and {n_val} for validation.')

print(f'init shape is {sem_cloud_features.shape}')
sem_train = sem_cloud_features[:n_train, :, :, :]
sem_val = sem_cloud_features[n_train:, :, :, :]
print(f'training shape is {sem_train.shape}, validation shape is {sem_val.shape}')

np.save(f'{export_ds}/sem_clouds3.npy', sem_train)
np.save(f'{export_ds}/sem_clouds_val_200.npy', sem_val)

print(f'Wrote features to {export_ds}/sem_clouds3.npy')
print(f'Wrote tiny features to {export_ds}/sem_clouds_val_200.npy')

we have in total 5000
we will keep 4500 for training and 500 for validation.
init shape is (5000, 2, 100, 100)
training shape is (4500, 2, 100, 100), validation shape is (500, 2, 100, 100)
Wrote features to /media/scratch/berlukas/nuscenes/sem_clouds3.npy
Wrote tiny features to /media/scratch/berlukas/nuscenes/sem_clouds_val_200.npy


In [9]:
sem_val_tiny = sem_val[:500, :, :, :]
np.save(f'{export_ds}/sem_clouds_val_tiny.npy', sem_val_tiny)